# Step 8: Transformer 아키텍처 - 현대 LLM의 기반

"Attention is All You Need" 논문에서 제안된 Transformer는 현재 모든 대형 언어 모델의 기반이 되는 아키텍처입니다.

## 학습 목표
1. Transformer의 전체 구조 이해
2. Encoder와 Decoder 구현
3. Layer Normalization과 Residual Connection
4. Feed-Forward Network
5. 완전한 Transformer 블록 구축

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from copy import deepcopy

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# 재현성
torch.manual_seed(42)
np.random.seed(42)

## 1. Transformer 아키텍처 개요

Transformer는 크게 두 부분으로 구성됩니다:
- **Encoder**: 입력을 처리하여 표현(representation)을 생성
- **Decoder**: Encoder의 출력을 받아 순차적으로 출력을 생성

In [ ]:
# Transformer 구조 시각화
def visualize_transformer_architecture():
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # 컴포넌트 정의
    components = {
        'input_embed': {'pos': (2, 1), 'size': (2, 0.8), 'color': 'lightblue', 'text': 'Input\nEmbedding'},
        'output_embed': {'pos': (6, 1), 'size': (2, 0.8), 'color': 'lightblue', 'text': 'Output\nEmbedding'},
        'pos_enc_1': {'pos': (2, 2.2), 'size': (2, 0.8), 'color': 'lightgreen', 'text': 'Positional\nEncoding'},
        'pos_enc_2': {'pos': (6, 2.2), 'size': (2, 0.8), 'color': 'lightgreen', 'text': 'Positional\nEncoding'},
        'encoder': {'pos': (2, 5), 'size': (2, 3), 'color': 'lightyellow', 'text': 'Encoder\n(Nx)'},
        'decoder': {'pos': (6, 5), 'size': (2, 3), 'color': 'lightcoral', 'text': 'Decoder\n(Nx)'},
        'output': {'pos': (6, 8.5), 'size': (2, 0.8), 'color': 'lightgray', 'text': 'Linear &\nSoftmax'}
    }
    
    # 컴포넌트 그리기
    for comp, props in components.items():
        rect = plt.Rectangle(props['pos'], props['size'][0], props['size'][1], 
                           facecolor=props['color'], edgecolor='black', linewidth=2)
        ax.add_patch(rect)
        ax.text(props['pos'][0] + props['size'][0]/2, props['pos'][1] + props['size'][1]/2, 
               props['text'], ha='center', va='center', fontsize=10, fontweight='bold')
    
    # 화살표 그리기
    arrows = [
        ((3, 1.8), (3, 2.2)),  # input embed → pos enc
        ((3, 3), (3, 5)),      # pos enc → encoder
        ((7, 1.8), (7, 2.2)),  # output embed → pos enc
        ((7, 3), (7, 5)),      # pos enc → decoder
        ((4, 6.5), (6, 6.5)),  # encoder → decoder
        ((7, 8), (7, 8.5))     # decoder → output
    ]
    
    for start, end in arrows:
        ax.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1], 
                head_width=0.2, head_length=0.2, fc='black', ec='black')
    
    # Encoder 내부 구조
    enc_components = [
        ('Multi-Head\nAttention', 3.5),
        ('Add & Norm', 4.5),
        ('Feed\nForward', 5.5),
        ('Add & Norm', 6.5)
    ]
    
    for i, (text, y) in enumerate(enc_components):
        ax.text(1.5, y, text, ha='center', va='center', fontsize=8, 
               bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.7))
    
    # Decoder 내부 구조
    dec_components = [
        ('Masked\nMulti-Head\nAttention', 3.5),
        ('Add & Norm', 4.2),
        ('Multi-Head\nAttention', 5),
        ('Add & Norm', 5.8),
        ('Feed\nForward', 6.6),
        ('Add & Norm', 7.4)
    ]
    
    for i, (text, y) in enumerate(dec_components):
        ax.text(8.5, y, text, ha='center', va='center', fontsize=8,
               bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.7))
    
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Transformer Architecture', fontsize=16, fontweight='bold')
    
    # 레이블
    ax.text(3, 0.5, 'Inputs', ha='center', fontsize=12)
    ax.text(7, 0.5, 'Outputs\n(shifted right)', ha='center', fontsize=12)
    ax.text(7, 9.5, 'Output\nProbabilities', ha='center', fontsize=12)
    
    plt.tight_layout()
    plt.show()

visualize_transformer_architecture()

## 2. Multi-Head Attention (복습 및 개선)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_heads == 0
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        # Q, K, V 프로젝션
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # 1. Q, K, V 계산 및 헤드 분리
        Q = self.W_q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        
        # 2. Attention 계산
        attn_output, attn_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # 3. 헤드 합치기
        attn_output = attn_output.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model
        )
        
        # 4. 최종 프로젝션
        output = self.W_o(attn_output)
        
        return output, attn_weights
    
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        output = torch.matmul(attn_weights, V)
        
        return output, attn_weights

# 테스트
d_model = 512
n_heads = 8
seq_len = 10
batch_size = 2

mha = MultiHeadAttention(d_model, n_heads)
x = torch.randn(batch_size, seq_len, d_model)
output, weights = mha(x, x, x)

print(f"입력 형태: {x.shape}")
print(f"출력 형태: {output.shape}")
print(f"Attention 가중치 형태: {weights.shape}")

## 3. Position-wise Feed-Forward Networks

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """Position-wise Feed-Forward Network"""
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        # FFN(x) = max(0, xW1 + b1)W2 + b2
        return self.fc2(self.dropout(self.activation(self.fc1(x))))

# FFN 동작 시각화
def visualize_ffn():
    d_model = 4
    d_ff = 8
    seq_len = 3
    
    # 예시 입력
    x = torch.randn(1, seq_len, d_model)
    
    # FFN 레이어
    ffn = PositionwiseFeedForward(d_model, d_ff)
    
    # 중간 활성화 추출
    with torch.no_grad():
        intermediate = ffn.activation(ffn.fc1(x))
        output = ffn.fc2(intermediate)
    
    # 시각화
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 입력
    im1 = axes[0].imshow(x[0].numpy(), cmap='coolwarm', aspect='auto')
    axes[0].set_title(f'Input\n({seq_len} × {d_model})')
    axes[0].set_ylabel('Position')
    axes[0].set_xlabel('d_model')
    plt.colorbar(im1, ax=axes[0])
    
    # 중간 표현
    im2 = axes[1].imshow(intermediate[0].numpy(), cmap='coolwarm', aspect='auto')
    axes[1].set_title(f'After First Linear + ReLU\n({seq_len} × {d_ff})')
    axes[1].set_ylabel('Position')
    axes[1].set_xlabel('d_ff')
    plt.colorbar(im2, ax=axes[1])
    
    # 출력
    im3 = axes[2].imshow(output[0].numpy(), cmap='coolwarm', aspect='auto')
    axes[2].set_title(f'Output\n({seq_len} × {d_model})')
    axes[2].set_ylabel('Position')
    axes[2].set_xlabel('d_model')
    plt.colorbar(im3, ax=axes[2])
    
    plt.suptitle('Position-wise Feed-Forward Network', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"FFN의 특징:")
    print(f"- 각 위치에 독립적으로 적용 (position-wise)")
    print(f"- 차원 확장 후 축소: {d_model} → {d_ff} → {d_model}")
    print(f"- 비선형성 추가로 모델의 표현력 증가")

visualize_ffn()

## 4. Layer Normalization과 Residual Connection

In [ ]:
class LayerNorm(nn.Module):
    """Layer Normalization"""
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

class ResidualConnection(nn.Module):
    """Residual Connection with Layer Normalization"""
    def __init__(self, size, dropout=0.1):
        super(ResidualConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, sublayer):
        """Apply residual connection to any sublayer with the same size."""
        return x + self.dropout(sublayer(self.norm(x)))

# Layer Norm 효과 시각화
def visualize_layer_norm():
    # 예시 데이터 생성 (분포가 다른 특징들)
    batch_size = 100
    seq_len = 50
    d_model = 4
    
    # 각 특징이 다른 스케일을 가지도록
    x = torch.randn(batch_size, seq_len, d_model)
    x[:, :, 0] *= 10  # 첫 번째 특징은 큰 스케일
    x[:, :, 1] *= 0.1  # 두 번째 특징은 작은 스케일
    x[:, :, 2] += 5  # 세 번째 특징은 큰 평균
    
    # Layer Normalization 적용
    layer_norm = LayerNorm(d_model)
    x_normalized = layer_norm(x)
    
    # 시각화
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # 원본 데이터 분포
    for i in range(4):
        axes[0, i].hist(x[:, :, i].flatten().numpy(), bins=50, alpha=0.7)
        axes[0, i].set_title(f'Original Feature {i+1}')
        axes[0, i].set_ylabel('Count')
        
        # 통계 정보
        mean = x[:, :, i].mean().item()
        std = x[:, :, i].std().item()
        axes[0, i].text(0.05, 0.95, f'μ={mean:.2f}\nσ={std:.2f}', 
                       transform=axes[0, i].transAxes, 
                       verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 정규화된 데이터 분포
    for i in range(4):
        axes[1, i].hist(x_normalized[:, :, i].flatten().detach().numpy(), bins=50, alpha=0.7, color='orange')
        axes[1, i].set_title(f'Normalized Feature {i+1}')
        axes[1, i].set_ylabel('Count')
        
        # 통계 정보
        mean = x_normalized[:, :, i].mean().item()
        std = x_normalized[:, :, i].std().item()
        axes[1, i].text(0.05, 0.95, f'μ={mean:.2f}\nσ={std:.2f}', 
                       transform=axes[1, i].transAxes, 
                       verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.suptitle('Layer Normalization 효과', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    print("Layer Normalization의 효과:")
    print("- 각 샘플의 특징들이 동일한 스케일로 정규화됨")
    print("- 학습 안정성 향상")
    print("- 더 빠른 수렴")

visualize_layer_norm()

## 5. Encoder Layer

In [ ]:
class EncoderLayer(nn.Module):
    """Transformer Encoder Layer"""
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Self-attention with residual connection and layer norm
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)
        
        # Feed-forward with residual connection and layer norm
        ff_output = self.feed_forward(x)
        x = x + self.dropout2(ff_output)
        x = self.norm2(x)
        
        return x

# Encoder Layer 동작 시각화
def visualize_encoder_layer():
    d_model = 128
    n_heads = 8
    d_ff = 512
    seq_len = 10
    
    # 입력 생성
    x = torch.randn(1, seq_len, d_model)
    
    # Encoder Layer
    encoder_layer = EncoderLayer(d_model, n_heads, d_ff)
    
    # 중간 출력 수집을 위한 hook
    intermediate_outputs = {}
    
    def get_activation(name):
        def hook(model, input, output):
            intermediate_outputs[name] = output.detach()
        return hook
    
    # Hook 등록
    encoder_layer.self_attn.register_forward_hook(get_activation('self_attn'))
    encoder_layer.norm1.register_forward_hook(get_activation('after_norm1'))
    encoder_layer.feed_forward.register_forward_hook(get_activation('feed_forward'))
    encoder_layer.norm2.register_forward_hook(get_activation('after_norm2'))
    
    # Forward pass
    output = encoder_layer(x)
    
    # 시각화
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.ravel()
    
    # 각 단계의 출력 norm 시각화
    stages = [
        ('Input', x),
        ('After Self-Attention', intermediate_outputs['self_attn'][0]),
        ('After Norm1', intermediate_outputs['after_norm1']),
        ('After Feed-Forward', intermediate_outputs['feed_forward']),
        ('After Norm2', intermediate_outputs['after_norm2']),
        ('Final Output', output)
    ]
    
    for i, (name, tensor) in enumerate(stages):
        # 각 위치의 벡터 norm 계산
        norms = tensor[0].norm(dim=-1).numpy()
        axes[i].bar(range(seq_len), norms)
        axes[i].set_title(name)
        axes[i].set_xlabel('Position')
        axes[i].set_ylabel('Vector Norm')
        axes[i].set_ylim(0, max(norms) * 1.2)
        
        # 평균 norm 표시
        avg_norm = norms.mean()
        axes[i].axhline(y=avg_norm, color='r', linestyle='--', alpha=0.5)
        axes[i].text(0.02, 0.98, f'Avg: {avg_norm:.2f}', 
                    transform=axes[i].transAxes, 
                    verticalalignment='top')
    
    plt.suptitle('Encoder Layer의 각 단계별 출력', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_encoder_layer()

## 6. Decoder Layer

In [ ]:
class DecoderLayer(nn.Module):
    """Transformer Decoder Layer"""
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.norm3 = LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        # Masked self-attention
        attn_output, _ = self.self_attn(x, x, x, tgt_mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)
        
        # Cross-attention (Query from decoder, Key/Value from encoder)
        attn_output, _ = self.cross_attn(x, encoder_output, encoder_output, src_mask)
        x = x + self.dropout2(attn_output)
        x = self.norm2(x)
        
        # Feed-forward
        ff_output = self.feed_forward(x)
        x = x + self.dropout3(ff_output)
        x = self.norm3(x)
        
        return x

# Decoder의 Attention 시각화
def visualize_decoder_attention():
    d_model = 128
    n_heads = 8
    d_ff = 512
    src_len = 8  # Encoder 시퀀스 길이
    tgt_len = 6  # Decoder 시퀀스 길이
    
    # 입력 생성
    encoder_output = torch.randn(1, src_len, d_model)
    decoder_input = torch.randn(1, tgt_len, d_model)
    
    # Causal mask for decoder self-attention
    tgt_mask = torch.triu(torch.ones(tgt_len, tgt_len), diagonal=1).bool()
    tgt_mask = ~tgt_mask  # Invert for attention
    
    # Decoder Layer
    decoder_layer = DecoderLayer(d_model, n_heads, d_ff)
    
    # Attention weights 추출을 위해 forward 수정
    self_attn_weights = None
    cross_attn_weights = None
    
    # Hook으로 attention weights 캡처
    def get_self_attn(module, input, output):
        nonlocal self_attn_weights
        self_attn_weights = output[1]
    
    def get_cross_attn(module, input, output):
        nonlocal cross_attn_weights
        cross_attn_weights = output[1]
    
    decoder_layer.self_attn.register_forward_hook(get_self_attn)
    decoder_layer.cross_attn.register_forward_hook(get_cross_attn)
    
    # Forward
    output = decoder_layer(decoder_input, encoder_output, tgt_mask=tgt_mask)
    
    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Self-attention (masked)
    if self_attn_weights is not None:
        # 첫 번째 헤드만 시각화
        sns.heatmap(self_attn_weights[0, 0].detach().numpy(), 
                   ax=axes[0], cmap='Blues', cbar=True,
                   xticklabels=[f'T{i}' for i in range(tgt_len)],
                   yticklabels=[f'T{i}' for i in range(tgt_len)])
        axes[0].set_title('Decoder Self-Attention\n(Masked)')
        axes[0].set_xlabel('Key/Value (Target)')
        axes[0].set_ylabel('Query (Target)')
    
    # Cross-attention
    if cross_attn_weights is not None:
        sns.heatmap(cross_attn_weights[0, 0].detach().numpy(), 
                   ax=axes[1], cmap='Reds', cbar=True,
                   xticklabels=[f'S{i}' for i in range(src_len)],
                   yticklabels=[f'T{i}' for i in range(tgt_len)])
        axes[1].set_title('Decoder Cross-Attention\n(Decoder → Encoder)')
        axes[1].set_xlabel('Key/Value (Source)')
        axes[1].set_ylabel('Query (Target)')
    
    plt.tight_layout()
    plt.show()
    
    print("Decoder의 두 가지 Attention:")
    print("1. Self-Attention: 이전에 생성된 토큰들만 참조 (Masked)")
    print("2. Cross-Attention: Encoder의 출력을 참조하여 소스 정보 활용")

visualize_decoder_attention()

## 7. Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Positional encoding 계산
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        # 주파수 계산
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * 
            -(math.log(10000.0) / d_model)
        )
        
        # 사인/코사인 적용
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

# Positional Encoding이 주는 효과 시각화
def demonstrate_positional_encoding():
    d_model = 512
    seq_len = 100
    
    # 동일한 임베딩을 가진 시퀀스 생성
    embedding = torch.randn(1, 1, d_model)
    repeated_embedding = embedding.repeat(1, seq_len, 1)
    
    # Positional Encoding 적용
    pos_encoder = PositionalEncoding(d_model, dropout=0)
    encoded = pos_encoder(repeated_embedding)
    
    # 서로 다른 위치의 임베딩 간 유사도 계산
    similarities = torch.zeros(seq_len, seq_len)
    for i in range(seq_len):
        for j in range(seq_len):
            # 코사인 유사도
            sim = F.cosine_similarity(
                encoded[0, i].unsqueeze(0), 
                encoded[0, j].unsqueeze(0)
            )
            similarities[i, j] = sim.item()
    
    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Positional Encoding 패턴
    pe_pattern = pos_encoder.pe[0, :50, :128].numpy()
    im1 = axes[0].imshow(pe_pattern.T, cmap='RdBu', aspect='auto')
    axes[0].set_title('Positional Encoding Pattern')
    axes[0].set_xlabel('Position')
    axes[0].set_ylabel('Dimension')
    plt.colorbar(im1, ax=axes[0])
    
    # 위치 간 유사도
    im2 = axes[1].imshow(similarities[:50, :50].numpy(), cmap='coolwarm', aspect='auto')
    axes[1].set_title('Position Similarity After Encoding')
    axes[1].set_xlabel('Position j')
    axes[1].set_ylabel('Position i')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
    print("Positional Encoding의 효과:")
    print("- 같은 임베딩이라도 위치에 따라 다른 표현을 가짐")
    print("- 가까운 위치일수록 더 유사한 인코딩")
    print("- 주기적 패턴으로 상대적 위치 정보 인코딩")

demonstrate_positional_encoding()

## 8. 완전한 Transformer 구축

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout) 
            for _ in range(n_layers)
        ])
        self.norm = LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class TransformerDecoder(nn.Module):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout) 
            for _ in range(n_layers)
        ])
        self.norm = LayerNorm(d_model)
        
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, n_heads=8, 
                 n_layers=6, d_ff=2048, max_len=5000, dropout=0.1):
        super(Transformer, self).__init__()
        
        # 임베딩 레이어
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len, dropout)
        
        # Transformer 블록
        self.encoder = TransformerEncoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = TransformerDecoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        # 출력 레이어
        self.output_projection = nn.Linear(d_model, tgt_vocab_size)
        
        # 가중치 초기화
        self._init_weights()
        
    def _init_weights(self):
        # Xavier 초기화
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # 소스 인코딩
        src_emb = self.positional_encoding(self.src_embedding(src) * math.sqrt(self.src_embedding.embedding_dim))
        encoder_output = self.encoder(src_emb, src_mask)
        
        # 타겟 디코딩
        tgt_emb = self.positional_encoding(self.tgt_embedding(tgt) * math.sqrt(self.tgt_embedding.embedding_dim))
        decoder_output = self.decoder(tgt_emb, encoder_output, src_mask, tgt_mask)
        
        # 출력 프로젝션
        output = self.output_projection(decoder_output)
        
        return output

# 모델 생성 및 파라미터 수 확인
def create_transformer_model():
    # 하이퍼파라미터
    src_vocab_size = 10000
    tgt_vocab_size = 10000
    d_model = 512
    n_heads = 8
    n_layers = 6
    d_ff = 2048
    
    # 모델 생성
    model = Transformer(
        src_vocab_size, tgt_vocab_size, 
        d_model, n_heads, n_layers, d_ff
    )
    
    # 파라미터 수 계산
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Transformer 모델 생성 완료!")
    print(f"\n모델 구성:")
    print(f"- 소스/타겟 어휘 크기: {src_vocab_size}")
    print(f"- 모델 차원 (d_model): {d_model}")
    print(f"- 헤드 수: {n_heads}")
    print(f"- 레이어 수: {n_layers}")
    print(f"- FFN 차원: {d_ff}")
    print(f"\n총 파라미터 수: {total_params:,}")
    print(f"학습 가능한 파라미터 수: {trainable_params:,}")
    
    # 각 컴포넌트별 파라미터 수
    print("\n컴포넌트별 파라미터 수:")
    for name, module in model.named_children():
        params = sum(p.numel() for p in module.parameters())
        print(f"- {name}: {params:,}")
    
    return model

model = create_transformer_model()

## 9. 간단한 번역 예제

In [ ]:
def create_padding_mask(seq, pad_idx=0):
    """패딩 마스크 생성"""
    return (seq != pad_idx).unsqueeze(1).unsqueeze(2)

def create_look_ahead_mask(size):
    """Look-ahead 마스크 생성 (미래 정보 차단)"""
    mask = torch.triu(torch.ones(size, size), diagonal=1).bool()
    return ~mask

# 간단한 번역 시뮬레이션
def translation_example():
    # 간단한 어휘
    src_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'hello': 3, 'world': 4}
    tgt_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, '안녕': 3, '세계': 4}
    
    # 예시 문장
    src_sentence = [src_vocab['<sos>'], src_vocab['hello'], src_vocab['world'], src_vocab['<eos>']]
    tgt_sentence = [tgt_vocab['<sos>'], tgt_vocab['안녕'], tgt_vocab['세계'], tgt_vocab['<eos>']]
    
    # 텐서로 변환
    src = torch.tensor([src_sentence])
    tgt = torch.tensor([tgt_sentence])
    
    # 마스크 생성
    src_mask = create_padding_mask(src)
    tgt_mask = create_look_ahead_mask(tgt.size(1))
    
    # 작은 모델 생성
    small_model = Transformer(
        src_vocab_size=5, tgt_vocab_size=5,
        d_model=64, n_heads=4, n_layers=2, d_ff=128
    )
    
    # Forward pass
    small_model.eval()
    with torch.no_grad():
        output = small_model(src, tgt[:, :-1], src_mask, tgt_mask[:3, :3])
    
    print("번역 예제:")
    print(f"소스: {src_sentence} → ['<sos>', 'hello', 'world', '<eos>']")
    print(f"타겟: {tgt_sentence} → ['<sos>', '안녕', '세계', '<eos>']")
    print(f"\n모델 출력 형태: {output.shape}")
    print(f"출력은 각 위치에서 다음 토큰의 확률 분포를 나타냅니다.")
    
    # 예측 확률 시각화
    probs = F.softmax(output[0], dim=-1).numpy()
    
    plt.figure(figsize=(8, 6))
    im = plt.imshow(probs, cmap='Blues', aspect='auto')
    plt.colorbar(im, label='Probability')
    plt.xlabel('Vocabulary Index')
    plt.ylabel('Position')
    plt.title('Output Probability Distribution')
    plt.xticks(range(5), ['<pad>', '<sos>', '<eos>', '안녕', '세계'])
    plt.yticks(range(3), ['After <sos>', 'After 안녕', 'After 세계'])
    plt.tight_layout()
    plt.show()

translation_example()

## 10. Transformer의 계산 효율성

In [ ]:
def analyze_transformer_complexity():
    seq_lengths = np.array([10, 50, 100, 500, 1000, 2000])
    d_model = 512
    
    # Self-attention 복잡도: O(n^2 * d)
    self_attention_complexity = seq_lengths**2 * d_model
    
    # FFN 복잡도: O(n * d * d_ff)
    d_ff = 2048
    ffn_complexity = seq_lengths * d_model * d_ff
    
    # RNN 복잡도 (비교용): O(n * d^2)
    rnn_complexity = seq_lengths * d_model**2
    
    # 시각화
    plt.figure(figsize=(12, 5))
    
    # 복잡도 비교
    plt.subplot(1, 2, 1)
    plt.plot(seq_lengths, self_attention_complexity/1e6, 'r-', label='Self-Attention', linewidth=2)
    plt.plot(seq_lengths, ffn_complexity/1e6, 'g-', label='Feed-Forward', linewidth=2)
    plt.plot(seq_lengths, rnn_complexity/1e6, 'b-', label='RNN', linewidth=2)
    plt.xlabel('Sequence Length')
    plt.ylabel('Computational Complexity (M)')
    plt.title('Computational Complexity Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 메모리 사용량
    plt.subplot(1, 2, 2)
    # Attention 메모리: O(n^2)
    attention_memory = seq_lengths**2
    # 모델 파라미터는 시퀀스 길이와 무관
    plt.plot(seq_lengths, attention_memory/1e6, 'r-', label='Attention Memory', linewidth=2)
    plt.xlabel('Sequence Length')
    plt.ylabel('Memory Usage (M elements)')
    plt.title('Memory Usage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    print("Transformer의 복잡도 분석:")
    print(f"- Self-Attention: O(n² × d) - 시퀀스 길이의 제곱에 비례")
    print(f"- Feed-Forward: O(n × d × d_ff) - 시퀀스 길이에 선형 비례")
    print(f"- 메모리: O(n²) - Attention 행렬 저장")
    print(f"\n장점:")
    print(f"- 병렬 처리 가능 (RNN과 달리)")
    print(f"- 장거리 의존성 직접 모델링")
    print(f"\n단점:")
    print(f"- 긴 시퀀스에서 메모리 문제")
    print(f"- 제곱 복잡도로 인한 계산 비용")

analyze_transformer_complexity()

## 11. 연습 문제

In [ ]:
# 문제 1: Encoder-only Transformer (BERT 스타일)
class EncoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff):
        super(EncoderOnlyTransformer, self).__init__()
        # TODO: 구현하기
        # 힌트: Encoder만 사용, [CLS] 토큰의 표현을 분류에 사용
        pass

# 문제 2: Decoder-only Transformer (GPT 스타일)
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff):
        super(DecoderOnlyTransformer, self).__init__()
        # TODO: 구현하기
        # 힌트: Decoder만 사용, Causal attention 필수
        pass

# 문제 3: Relative Position Encoding
def relative_position_encoding(seq_len, d_model, max_relative_position=128):
    # TODO: 구현하기
    # 힌트: 절대 위치가 아닌 상대 위치를 인코딩
    pass

## 정리

이번 튜토리얼에서 배운 내용:
1. **Transformer 아키텍처**: Encoder-Decoder 구조
2. **Multi-Head Attention**: 병렬로 여러 관점 학습
3. **Position-wise FFN**: 각 위치에 독립적으로 적용
4. **Layer Normalization**: 학습 안정성
5. **Residual Connection**: 깊은 네트워크 학습 가능
6. **Positional Encoding**: 순서 정보 추가

### Transformer의 혁신성:
- **병렬 처리**: 모든 위치를 동시에 처리
- **장거리 의존성**: 직접적인 연결로 정보 전달
- **확장성**: 모델 크기를 키워 성능 향상

### 주요 변형:
- **BERT**: Encoder-only, 양방향 문맥 이해
- **GPT**: Decoder-only, 자기회귀적 생성
- **T5**: Encoder-Decoder, 모든 NLP 작업을 텍스트 생성으로

다음 단계에서는 이 Transformer를 기반으로 미니 GPT를 직접 구현해보겠습니다!